<a href="https://colab.research.google.com/github/detektor777/colab_list/blob/main/InstColorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Перейити в гугл диск и загрузить в корень диска файл с видео

https://drive.google.com/drive/

In [ ]:
#@title ##**Имя видео файла** { display-mode: "form" }
file_name = input("Введите имя файла: ") or "input.mp4"

In [ ]:
#@title ##**Проверка GPU** { display-mode: "form" }
!nvidia-smi

In [ ]:
#@title ##**Подключение к гугл диску** { display-mode: "form" }
from google.colab import drive
drive.mount('/mnt/gdrive')


In [ ]:
#@title ##**Создать папки для изображений** { display-mode: "form" }

import os
import shutil

color_upload_folder = '/mnt/gdrive/MyDrive/color_input'
color_result_folder = '/mnt/gdrive/MyDrive/color_output'


In [ ]:
#@title ##**Очистить входящую папку** { display-mode: "form" }
if os.path.isdir(color_upload_folder):
    shutil.rmtree(color_upload_folder)
os.makedirs(color_upload_folder)

In [ ]:
#@title ##**Очистить исходящую папку** { display-mode: "form" }
if os.path.isdir(color_result_folder):
    shutil.rmtree(color_result_folder)
os.makedirs(color_result_folder)

In [ ]:
#@title ##**Секвенция видео** { display-mode: "form" }
#если нужно продолжить предедущую обработку, то пропустите этот пункт
import cv2
import imageio
import os
import tqdm
import subprocess
import numpy as np
import time

color_upload_folder = '/mnt/gdrive/MyDrive/color_input'
color_result_folder = '/mnt/gdrive/MyDrive/color_output'

library = "imageio" #@param ["cv2","pyav","imageio","ffmpeg","skvideo","scipy","moviepy"]
delay = "0.1" #@param [0, 0.01, 0.05, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
if (library == "cv2"):
    path = '/mnt/gdrive/MyDrive/'
    full_path = os.path.join(path, file_name)

    cmd = 'ffprobe -v error -select_streams v:0 -count_packets -show_entries stream=nb_read_packets -of csv=p=0 ' + full_path
    output = subprocess.check_output(cmd, shell=True).decode('utf-8').strip()

    cmd = 'ffprobe -v error -select_streams v:0 -show_entries stream=r_frame_rate -of csv=p=0 ' + full_path
    output = subprocess.check_output(cmd, shell=True).decode('utf-8').strip()
    vidcap = cv2.VideoCapture(full_path)
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

    duration = frame_count / fps

    print("FPS: ", fps)
    print("Duration: ", duration)
    print("Frames: ", frame_count)

    vidcap = cv2.VideoCapture(full_path)
    success, image = vidcap.read()
    
    tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
    pbar_cv2 = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
    while success:
        try:
            cv2.imwrite(f"/mnt/gdrive/MyDrive/color_input/frame{int(vidcap.get(cv2.CAP_PROP_POS_FRAMES)):09d}.jpg", image)
        except cv2.error:
            print("Error writing to disk. Retrying...")
            continue
        pbar_cv2.update(1)
        time.sleep(float(delay))
        success, image = vidcap.read()

    pbar_cv2.close()
    vidcap.release()

elif (library == "pyav"):
    !pip install av
    import av
    video = av.open(full_path)

    fps_value = video.streams.video[0].average_rate
    frame_count = video.streams.video[0].frames

    if fps_value is not None:
        duration = frame_count / fps_value

        print("FPS: ", fps_value)
        print("Duration: ", duration)
        print("Frames: ", frame_count)

        tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
        pbar_pyav = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
        for packet in video.demux():
            for frame in packet.decode():
                img = frame.to_ndarray(format='rgb24')
                img = img.astype('uint8')  # преобразование к типу uint8
                img_path = f"/mnt/gdrive/MyDrive/color_input/frame{packet.pts:09d}.jpg"
                imageio.imwrite(img_path, img)
                pbar_pyav.update(1)
                time.sleep(float(delay))

        pbar_pyav.close()
        video.close()
    else:
        print("Error: FPS value is None.")


elif (library == "imageio"):
    path = '/mnt/gdrive/MyDrive/'
    full_path = os.path.join(path, file_name)

    video = imageio.get_reader(full_path)

    fps = video.get_meta_data()['fps']
    frame_count = video.count_frames()
    duration = frame_count / fps

    print("FPS: ", fps)
    print("Duration: ", duration)
    print("Frames: ", frame_count)

    tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
    pbar_imageio = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
    for i, frame in enumerate(video):
      img_path = f"/mnt/gdrive/MyDrive/color_input/frame{i+1:09d}.jpg"
      if os.path.isfile(img_path):
          pbar_imageio.update(1)
          continue
      while True:
          try:
              imageio.imwrite(img_path, frame)
          except Exception as e:
              print(f"Error writing to disk: {str(e)}. Retrying...")
              continue
          break
      pbar_imageio.update(1)
      time.sleep(float(delay))

    pbar_imageio.close()
    video.close()

elif (library == "ffmpeg"):
    !pip install ffmpeg-python
    import ffmpeg
    path = '/mnt/gdrive/MyDrive/'
    full_path = os.path.join(path, file_name)

    probe = ffmpeg.probe(full_path)
    video_info = next(stream for stream in probe['streams'] if stream['codec_type'] == 'video')
    fps = video_info['r_frame_rate']
    duration = float(video_info['duration'])
    frame_count = int(video_info['nb_frames'])

    print("FPS: ", fps)
    print("Duration: ", duration)
    print("Frames: ", frame_count)

    pbar_ffmpeg = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
    process = (
        ffmpeg
        .input(full_path)
        .output('pipe:', format='rawvideo', pix_fmt='rgb24', qscale=0)
        .run_async(pipe_stdout=True)
    )

    for i in range(frame_count):
        try:
            raw_video = process.stdout.read(video_info['width'] * video_info['height'] * 3)
            frame = np.frombuffer(raw_video, dtype='uint8').reshape((video_info['height'], video_info['width'], 3))
            frame_path = f"/mnt/gdrive/MyDrive/color_input/frame{i:09d}.jpg"
            if os.path.isfile(frame_path):
              pbar_ffmpeg.update(1)
              continue
            imageio.imwrite(frame_path, frame)
        except Exception as e:
            print(f"Error writing to disk: {str(e)}. Retrying...")
            continue
        pbar_ffmpeg.update(1)
        time.sleep(float(delay))

    pbar_ffmpeg.close()
    process.wait()
elif (library == "skvideo"):
    !pip install scikit-video
    import skvideo.io
    path = '/mnt/gdrive/MyDrive/'
    full_path = os.path.join(path, file_name)

    video = skvideo.io.vread(full_path)

    fps_str = skvideo.io.ffprobe(full_path)['video']['@avg_frame_rate']
    fps_parts = fps_str.split('/')
    fps = int(fps_parts[0]) / int(fps_parts[1])
    frame_count = len(video)
    duration = frame_count / fps

    print("FPS: ", fps)
    print("Duration: ", duration)
    print("Frames: ", frame_count)

    tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
    outputdict = {'-q:1': '1'}
    pbar_skvideo = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
    for i, frame in enumerate(video):
        while True:
            try:
                skvideo.io.vwrite(f"/mnt/gdrive/MyDrive/color_input/frame{i:09d}.jpg", frame, outputdict=outputdict)
            except Exception as e:
                print(f"Error writing to disk: {str(e)}. Retrying...")
                continue
            break
        pbar_skvideo.update(1)
        time.sleep(float(delay))

    pbar_skvideo.close()

elif (library == "scipy"):
    import scipy.misc
    import scipy.ndimage
    
    full_path = os.path.join(path, file_name)
    video = imageio.get_reader(full_path)
    fps_value = video.get_meta_data()['fps']
    frame_count = video.get_length()

    if fps_value is not None:
        duration = frame_count / fps_value

        print("FPS: ", fps_value)

        tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
        pbar_scipy = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
        for i, im in enumerate(video):
            img_path = f"/mnt/gdrive/MyDrive/color_input/frame{i+1:09d}.jpg"
            im = Image.fromarray(im)
            im.save(img_path, quality=100)
            pbar_scipy.update(1)
            time.sleep(float(delay))

        pbar_scipy.close()
    else:
        print("Error: FPS value is None.")

elif library == "moviepy":
    from moviepy.editor import VideoFileClip
    from PIL import Image
    full_path = os.path.join(path, file_name)
    video = VideoFileClip(full_path)
    fps_value = video.fps
    frame_count = int(video.duration * fps_value)

    print("FPS: ", fps_value)

    tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
    pbar_moviepy = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
    for i, frame in enumerate(video.iter_frames()):
        img_path = f"/mnt/gdrive/MyDrive/color_input/frame{i+1:09d}.jpg"
        im = Image.fromarray(frame)
        im.save(img_path, quality=100)
        pbar_moviepy.update(1)
        time.sleep(float(delay))

    pbar_moviepy.close()

In [ ]:
#@title ##**Инсталяция** { display-mode: "form" }
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install dominate==2.4.0
!pip install detectron2==0.1.2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html
!pip install torch torchvision
!pip install cython
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
#@title ##**Клонирование репозитория** { display-mode: "form" }
!git clone https://github.com/ericsujw/InstColorization.git
%cd /content/InstColorization

In [ ]:
#@title ##**Копирование с гугл диска** { display-mode: "form" }
import shutil
import time
input_dir_bbox = "example_bbox"

if os.path.isdir(input_dir_bbox):
    shutil.rmtree(input_dir_bbox)
os.makedirs(input_dir_bbox)

example_bbox_folder = "/content/InstColorization/example"
input_bbox_folder = "/mnt/gdrive/MyDrive/color_input"

if os.path.isdir(example_bbox_folder):
    shutil.rmtree(example_bbox_folder)
os.makedirs(example_bbox_folder)

for file_name_image in os.listdir(input_bbox_folder):
    input_path = os.path.join(input_bbox_folder, file_name_image)
    output_path = os.path.join(example_bbox_folder, file_name_image)
    success = False
    while not success:
        try:
            shutil.copy(input_path, output_path)
            success = True
        except Exception as e:
            print(f"Error copying file {input_path} to {output_path}: {str(e)}")
            time.sleep(5) # wait 5 seconds before trying again


In [ ]:
#@title ##**Загрузка модели** { display-mode: "form" }
!gdown https://drive.google.com/uc?id=1Xb-DKAA9ibCVLqm8teKd1MWk6imjwTBh
!unzip checkpoints.zip -d /content/InstColorization

In [ ]:
#@title ##**Обнаружение объектов** { display-mode: "form" }
from os.path import join, isfile, isdir
from os import listdir
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from argparse import ArgumentParser



import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import cv2

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg

import torch

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

In [ ]:
#@title ##**Создать папку** { display-mode: "form" }
input_dir = "example"

image_list = [f for f in listdir(input_dir) if isfile(join(input_dir, f))]
output_npz_dir = "{0}_bbox".format(input_dir)
if os.path.isdir(output_npz_dir) is False:
    print('Create path: {0}'.format(output_npz_dir))
    os.makedirs(output_npz_dir)

In [ ]:
#@title ##**Установки входных данных и наполнение папки example_bbox** { display-mode: "form" }
for image_path in image_list:
    img = cv2.imread(join(input_dir, image_path))
    lab_image = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l_channel, a_channel, b_channel = cv2.split(lab_image)
    l_stack = np.stack([l_channel, l_channel, l_channel], axis=2)
    outputs = predictor(l_stack)
    save_path = join(output_npz_dir, image_path.split('.')[0])
    pred_bbox = outputs["instances"].pred_boxes.to(torch.device('cpu')).tensor.numpy()
    pred_scores = outputs["instances"].scores.cpu().data.numpy()
    np.savez(save_path, bbox = pred_bbox, scores = pred_scores)

In [ ]:
#@title ##**Установка библиотек** { display-mode: "form" }
import sys
import time
from options.train_options import TestOptions
from models import create_model

import torch
from tqdm import tqdm_notebook

from fusion_dataset import Fusion_Testing_Dataset
from util import util
import multiprocessing
multiprocessing.set_start_method('spawn', True)

torch.backends.cudnn.benchmark = True

sys.argv = [sys.argv[0]]
opt = TestOptions().parse()

In [ ]:
#@title ##**Создать папку для результатов** { display-mode: "form" }

save_img_path = '/mnt/gdrive/MyDrive/color_output'
if os.path.isdir(save_img_path) is False:
    print('Create path: {0}'.format(save_img_path))
    os.makedirs(save_img_path)
opt.batch_size = 1
dataset = Fusion_Testing_Dataset(opt, -1)
dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=opt.batch_size)

dataset_size = len(dataset)
print('#Testing images = %d' % dataset_size)
!ls example_bbox

In [ ]:
#@title ##**Установить модель** { display-mode: "form" }
model = create_model(opt)
model.setup_to_test('coco_finetuned_mask_256_ffs')

In [ ]:
#@title ##**Колоризация** { display-mode: "form" }
count_empty = 0

for data_raw in tqdm_notebook(dataset_loader):
    data_raw['full_img'][0] = data_raw['full_img'][0].cuda()
    if data_raw['empty_box'][0] == 0:
        data_raw['cropped_img'][0] = data_raw['cropped_img'][0].cuda()
        box_info = data_raw['box_info'][0]
        box_info_2x = data_raw['box_info_2x'][0]
        box_info_4x = data_raw['box_info_4x'][0]
        box_info_8x = data_raw['box_info_8x'][0]
        cropped_data = util.get_colorization_data(data_raw['cropped_img'], opt, ab_thresh=0, p=opt.sample_p)
        full_img_data = util.get_colorization_data(data_raw['full_img'], opt, ab_thresh=0, p=opt.sample_p)
        model.set_input(cropped_data)
        model.set_fusion_input(full_img_data, [box_info, box_info_2x, box_info_4x, box_info_8x])
        model.forward()
    else:
        count_empty += 1
        full_img_data = util.get_colorization_data(data_raw['full_img'], opt, ab_thresh=0, p=opt.sample_p)
        model.set_forward_without_box(full_img_data)
    model.save_current_imgs(join(save_img_path, data_raw['file_id'][0] + '.png'))
print('{0} images without bounding boxes'.format(count_empty))

In [ ]:
#@title ##**Очистить папку результа** { display-mode: "form" }
result_path = '/mnt/gdrive/MyDrive/color_output_results'
if os.path.isdir(result_path):
    shutil.rmtree(result_path)
os.makedirs(result_path)

In [ ]:
#@title ##**Запись результата секвенции** { display-mode: "form" }
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

result_path = '/mnt/gdrive/MyDrive/color_output_results'

# Получаем список всех файлов в папке 'examples'
folder_path = '/content/InstColorization/example'
img_name_list = [f[:-4] for f in os.listdir(folder_path) if f.endswith('.jpg')]

for img_name in img_name_list:
    img = cv2.imread(f'{folder_path}/{img_name}.jpg')
    lab_image = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l_channel, _, _ = cv2.split(lab_image)

    img = cv2.imread(f'/mnt/gdrive/MyDrive/color_output/{img_name}.png')
    if img is None:
        print(f"Image {img_name}.png not found or cannot be read.")
        continue

    lab_image = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    _, a_pred, b_pred = cv2.split(lab_image)
    a_pred = cv2.resize(a_pred, (l_channel.shape[1], l_channel.shape[0]))
    b_pred = cv2.resize(b_pred, (l_channel.shape[1], l_channel.shape[0]))
    gray_color = np.ones_like(a_pred) * 128

    gray_image = cv2.cvtColor(np.stack([l_channel, gray_color, gray_color], 2), cv2.COLOR_LAB2BGR)
    color_image = cv2.cvtColor(np.stack([l_channel, a_pred, b_pred], 2), cv2.COLOR_LAB2BGR)

    #cv2.imwrite(f'/mnt/gdrive/MyDrive/color_output_results/{img_name}.jpg', color_image)
    save_img_path = os.path.join(result_path, f'{img_name}.jpg')
    cv2.imwrite(save_img_path, color_image)

In [ ]:
#@title ##**Сборка видео** { display-mode: "form" }
import cv2
import os
from tqdm.notebook import tqdm

output_file = "/mnt/gdrive/MyDrive/output.mp4"
path = '/mnt/gdrive/MyDrive/'

full_path_file = os.path.join(path, file_name)
print(file_name)
print(full_path_file)


cap = cv2.VideoCapture(full_path)
#fps_of_video = int(cap.get(cv2.CAP_PROP_FPS))
vid = imageio.get_reader(full_path_file, 'ffmpeg')
fps_of_video = vid.get_meta_data()['fps']
cap.release()
print(fps_of_video)


img_files = os.listdir(result_path)
img_files.sort()

frame = cv2.imread(os.path.join(result_path, img_files[0]))
height, width, _ = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(output_file, fourcc, fps_of_video, (width, height))

for img_file in tqdm(img_files, desc='Сборка видео'):
    img_path = os.path.join(result_path, img_file)
    frame = cv2.imread(img_path)

    writer.write(frame)

writer.release()
print('Сборка завершена')